In [23]:
import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm
from bs4 import BeautifulSoup

class NinisiteTopicCrawler():
    """
    A class for crawling and extracting data from a NiniSite topic.
    """
    def __init__(self, topic_id):
        """
        Initializes the NinisiteTopicCrawler instance.

        Args:
            topic_id (int): The ID of the topic to crawl.
        """
        self.topic_id = topic_id
        self.main_link = 'https://www.ninisite.com'
        self.topic_link = self.main_link + '/discussion/topic/' + str(topic_id)
        self.topic_html = requests.get(self.topic_link).text
        self.topic_soup = BeautifulSoup(self.topic_html, 'html.parser')

    def topic_parser(self):
        """
        Parses the main topic page and extracts topic details.
        """
        soup = self.topic_soup.find('article', {'id': 'topic'})
        self.topic_detail = {
            'id': self.topic_id,
            'title': soup.find('div', {'class': 'col-xs-12 m-b-1 p-x-1 forum__topic--header'}).find('a').text,
            'text': soup.find('div', {'class': 'post-message topic-post__message col-xs-12 fr-view m-b-1 p-x-1'}).text,
            'date': soup.find('span', {'class': 'date'}).text,
            'username': soup.find('a', {'class': 'col-xs-9 col-md-12 text-md-center text-xs-right nickname'})['href'].split("/")[3],
            'userid': soup.find('a', {'class': 'col-xs-9 col-md-12 text-md-center text-xs-right nickname'})['href'].split("/")[2],
            'number_comments': int(soup.find_all('span', {'class': 'pull-xs-right'})[-1].text.split()[1])
        }
        self.topic_detail['number_pages'] = int(np.ceil(self.topic_detail['number_comments'] / 20))

    def comment_parser(self, comment):
        """
        Parses a comment element and extracts comment details.

        Args:
            comment (BeautifulSoup.Tag): The BeautifulSoup tag representing a comment.

        Returns:
            dict: Comment details including id, text, date, username, userid, and isReply.
        """
        self.comment = comment
        comment_detail = {
            'id': comment['id'].split('-')[-1],
            'text': comment.find('div', class_='post-message').text.strip(),
            'date': comment.find('span', class_='date').text.strip(),
            'username': comment.find('a', class_='col-xs-9 col-md-12 text-md-center text-xs-right nickname').text.strip(),
            'userid': comment.find('a', class_='col-xs-9 col-md-12 text-md-center text-xs-right nickname')['href'].split('/')[2],
            'isReply': bool(comment.find('div', {'class': 'reply-message'}))
        }
        return comment_detail

    def crawl_comments_in_topic(self, save_topic_detail = True, save_topic_comments = True):
        """
        Crawls all comments in the topic and returns a DataFrame containing the comment data.

        Returns:
            pd.DataFrame: DataFrame containing the comment data.
        """
        self.topic_parser()
        if save_topic_detail:
            pd.DataFrame(self.topic_detail,index=['topic_main']).to_csv(f'{self.topic_id}_main_post.csv')
            
        comments_data = []
        with tqdm(total=20*len(range(1, self.topic_detail['number_pages'] + 1))) as pbar:
            for pg_number in range(1, self.topic_detail['number_pages'] + 1):
                html = requests.get(self.topic_link + f'?page={pg_number}').text
                soup = BeautifulSoup(html, 'html.parser')
                self.comments = soup.find_all('article', class_='topic-post')
                for comment in self.comments[1:]:
                    comments_data.append(self.comment_parser(comment))
                    pbar.update(1)
                    time.sleep(1.5/20)

        topic_comments = pd.DataFrame(comments_data)
        if save_topic_comments:
            topic_comments.to_csv(f'{self.topic_id}_comments.csv')
        return pd.DataFrame(comments_data)


In [24]:
crawler = NinisiteTopicCrawler(11176867)

In [25]:
crawler.crawl_comments_in_topic()

 68%|███████████████████████████████████████████████████████▎                          | 27/40 [00:03<00:01,  6.92it/s]


,id,text,date,username,userid,isReply
0,292738424,همه اینجوری نیستن بعضیا اصالت انسان بودن خودش...,1402/03/05,m_mvi,3eaaac2b-4220-405b-af08-7e187b57ec6a,False
1,292738512,هیچ ربطی به مد نداره هرکسی به چیزی علاقه داره ...,1402/03/05,رقصءقیچی۲,2c6f48c7-2d03-413b-9420-da1b85907df5,False
2,292738571,خیلی کمن خیلیا من دیگه نمیبینم,1402/03/05,0zoha0,1bf45778-ac0c-40a0-b3bd-6cd80efdb233,True
3,292738581,وا کی میگ درس و معنویات مهم نی بهش انسانیت یاد...,1402/03/05,رفیق_ناباب_منم,e433cde0-e444-4be2-8cec-b857d1730d2f,False
4,292738743,تو جمع های اشتباهی قرار نگیرینجامعه ما به اون ...,1402/03/05,مهشیددد,f24c17ca-485a-4547-97c9-a548d169a56e,False
5,292738881,تو مدرسه و کلاسهایی که میره هیچ بچه ای دنبال ا...,1402/03/05,0zoha0,1bf45778-ac0c-40a0-b3bd-6cd80efdb233,True
6,292738923,قرار نیست زندگی و ادمها و علم و اداب همیشه به ...,1402/03/05,رقصءقیچی۲,2c6f48c7-2d03-413b-9420-da1b85907df5,True
7,292739137,نیستن من خیلی تو اجتماعم دایره ارتباطاتم زیاده...,1402/03/05,0zoha0,1bf45778-ac0c-40a0-b3bd-6cd80efdb233,True
8,292739180,من خودم قصد مادر شدن ندارم ولی اگه ی روزی بچه ...,1402/03/05,رفیق_ناباب_منم,e433cde0-e444-4be2-8cec-b857d1730d2f,True
9,292739230,بهتره خودتون تو یه جمع های بهتری قرار بدین. فر...,1402/03/05,من_باخودم_غریبه_ام,dbcfcbd8-6c0b-4fff-99ca-ee8707e6b9d7,True


In [17]:
pd.DataFrame(my_dict,index=['it'])

,id,title,text,date,username,userid,number_comments,number_pages
it,11176867,چجوری با این دنیای جدید کنار بیایم دیگه معنویا...,\nپس بچه هامون چی یادشون بدیم... الان فقط عمل ...,دیروز,0zoha0,1bf45778-ac0c-40a0-b3bd-6cd80efdb233,26,2
